In [21]:
import pydantic
from pydantic import BaseModel
from pydantic import ValidationError
from typing import Optional
import string

In [5]:
print(pydantic.__version__) # checking the version of pydantic

2.8.0


In [6]:
# this is our model

class Person(BaseModel):
    first_name: str
    last_name: str
    age: int


p = Person(first_name="John", last_name="Smith", age=42)
print(p)

first_name='John' last_name='Smith' age=42


In [7]:
try:
    Person(first_name="john", last_name="Smith", age="junk")

except ValidationError as ex:
    print(ex)

# above will show an error because we set int to str the age

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='junk', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/int_parsing


In [8]:
try:
    Person(first_name="john", last_name="Smith", age=42)

except ValidationError as ex:
    print(ex)


In [9]:
print(p.first_name)
print(p.first_name=="James")
print(p.age=="qwer")

John
False
False


### Validation Exceptions

In [11]:
try:
    Person(first_name="john", last_name="Smith", age="asdf")

except ValidationError as ex:
    exceptions = ex

In [12]:
exceptions.errors()

[{'type': 'int_parsing',
  'loc': ('age',),
  'msg': 'Input should be a valid integer, unable to parse string as an integer',
  'input': 'asdf',
  'url': 'https://errors.pydantic.dev/2.8/v/int_parsing'}]

In [13]:
exceptions.json()

'[{"type":"int_parsing","loc":["age"],"msg":"Input should be a valid integer, unable to parse string as an integer","input":"asdf","url":"https://errors.pydantic.dev/2.8/v/int_parsing"}]'

### Deserializing Data

In [15]:
data = {
    "first_name": "John",
    "last_name": "Smith",
    "age": 42
}

In [16]:
p = Person.model_validate(data)
p

Person(first_name='John', last_name='Smith', age=42)

In [18]:
data_json = '''
{
    "first_name": "John",
    "last_name": "Smith",
    "age": 42
}
'''

In [19]:
p = Person.model_validate_json(data_json)
p

Person(first_name='John', last_name='Smith', age=42)

### Required vs Optional Fields

By defult all fields are required, so if we pass only one values anyone like only age or first name or last name then this is raise validation error because by defult all the fields are must required. Below is the example for this point we do not provide first name and last name and only provide age so this is raising validation errors.

In [20]:
try:
    Person(age=42)

except ValidationError as ex:
    print(ex)

2 validation errors for Person
first_name
  Field required [type=missing, input_value={'age': 42}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
last_name
  Field required [type=missing, input_value={'age': 42}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing


Now how we can set some fields or any fields to be optional. so we have to make changes in pur model. model is a class which is given below. we need to add some defaults to make them optional.

In [23]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int = 0

In [24]:
Person.model_fields

{'first_name': FieldInfo(annotation=str, required=True),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=False, default=0)}

In [25]:
p = Person(first_name="John", last_name="Smith")
p

Person(first_name='John', last_name='Smith', age=0)

In [26]:
p = Person(first_name="John", last_name="Smith", age=42)
p

Person(first_name='John', last_name='Smith', age=42)

now the above two cells, if we do not define age then it will set it automatically to default value.

### Nullable Fields

while setting the default values, one must needs to be careful about the data type, like if age is int then we must have to set default in int as well similarly first name is in str then we have to set its default to str as well

In [27]:
class Person(BaseModel):
    first_name: str | None = None  # this is available in 3.10 versions and above
    last_name: str
    age: int = 0

In [28]:
Person.model_fields

{'first_name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=False, default=0)}

In [29]:
p = Person(last_name="Smith")
p

Person(first_name=None, last_name='Smith', age=0)

In [30]:
class Person(BaseModel):
    first_name: Optional[str] = None  # this means that it can be str or none like the work optional suggested
    last_name: str
    age: int = 0

In [31]:
p = Person(last_name="Smith")
p

Person(first_name=None, last_name='Smith', age=0)

In [32]:
class Person(BaseModel):
    first_name: str | None = None
    last_name: str
    age: int = 0
    lucky_numbers: list[int] = []

In [33]:
Person.model_fields

{'first_name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=False, default=0),
 'lucky_numbers': FieldInfo(annotation=list[int], required=False, default=[])}

In [34]:
p = Person(last_name="Smith", lucky_numbers=[1, "2", 3.0])
p

Person(first_name=None, last_name='Smith', age=0, lucky_numbers=[1, 2, 3])